In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_height = 224
img_width = 224
batchsize  = 32

test_datagen = ImageDataGenerator(rescale=1./255)               # no data augmentation performed, only scaling from 0 to 1

train_generator = test_datagen.flow_from_directory(
        r'/notebooks/Kaggle_FinalData_Preprocessed_V3/train',
        target_size=(img_height, img_width),
        batch_size=batchsize,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        r'/notebooks/Kaggle_FinalData_Preprocessed_V3/validation',
        target_size=(img_height, img_width),
        batch_size=batchsize,
        class_mode='categorical')

In [ ]:
train_img, labels = next(train_generator) 

In [ ]:
def plotImage(images_arr):
    fig, axes = plt.subplots(2, 5, figsize=(16,8))
    axes = axes.flatten()
    for img, img_y, ax in zip(images_arr, labels, axes):
        ax.imshow(img)
        ax.set_title('Severity {}'.format(np.argmax(img_y, -1)))
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImage(train_img)
#print(labels)  # uncomment this to check if the severity is correct.
print(train_img.shape[1:])

## Building the InceptionV3 Model with Dropouts and more Dense layers

In [ ]:
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Input, GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda

In [ ]:
input_shape = (224, 224, 3)
inception = InceptionV3(include_top=False, weights='imagenet', 
                                     input_shape=input_shape) 
num_class = 5

In [ ]:
inception.summary()

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
layers = [(layer, layer.name, layer.trainable) for layer in inception.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])   

In [ ]:
t_x, t_y = next(train_generator)

in_lay = Input(t_x.shape[1:])

add_model = Sequential()
add_model.add(inception)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.1))

In [ ]:
add_model.add(Dense(512, activation='relu', name = "dense_pre"))
add_model.add(Dropout(0.4))   #TSSL
add_model.add(Flatten())
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dense(num_class, activation='softmax', name = "prediction"))
model=add_model

model.summary()

In [ ]:
model.summary()

## Adding more Performance Metrics before compiling
### Source: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Built In Performance Metrics on Tensorflow

In [ ]:
import tensorflow_addons as tfa
from tensorflow_addons.metrics import CohenKappa
from keras.metrics import AUC, TopKCategoricalAccuracy, SpecificityAtSensitivity, Precision, Recall
cohen = tfa.metrics.CohenKappa(num_classes=5)
cohen_linear = tfa.metrics.CohenKappa(num_classes=5, weightage='linear')
cohen_quad = tfa.metrics.CohenKappa(num_classes=5, weightage='quadratic')


In [ ]:
from keras import optimizers
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizers.SGD(lr=1e-3),
  metrics=['accuracy','categorical_accuracy',f1_m,recall_m,precision_m,
                       SpecificityAtSensitivity(0.95), AUC(),cohen_quad]
)

## Keras Callbacks 

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights_v2.best.hdf5".format('InceptionV3_TSSL_V1_test')
filepath = r'/notebooks/Saved Weights/InceptionV3/weight_path'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='min', min_delta=0.01, cooldown=5, min_lr=0.00001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min",
                      min_delta=0.03,
                      patience=30) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, reduceLROnPlat, early]

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batchsize,
                    epochs=50,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.samples // batchsize,
                    verbose=1,
                    callbacks = callbacks_list
                    )           

## Plotting Performance

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('InceptionV3_TSSL_V1_test', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

ax1.plot(acc, label='Train Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(loss, label='Train Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

f.savefig(r'/notebooks/Saved Plots/Accuracy_Loss/InceptionV3/InceptionV3_TSSL_V1_test')

In [ ]:
model.save(r'/notebooks/Saved Models/InceptionV3/InceptionV3_TSSL_V1_test.h5/')   
print('Saved Model to disk')

## Evaluate the model using the test dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_height = 224
img_width = 224
batchsize = 32
eval_data_dir = r'/notebooks/Kaggle_FinalData_Preprocessed_V3/validation'
eval_datagen = ImageDataGenerator(rescale=1./255) # set validation split

eval_generator = eval_datagen.flow_from_directory(
    eval_data_dir,
    target_size=(img_height, img_width),
    batch_size=batchsize,
    class_mode='categorical',
    shuffle=False) 

filenames = eval_generator.filenames
nb_samples = len(filenames)




In [ ]:
eval_img, labels = next(eval_generator)
plotImage(eval_img)
#print(labels)  # uncomment this to check if the severity is correct.
print(eval_img.shape[1:])

In [ ]:
%%time
predictions = model.predict(eval_generator,nb_samples)

In [ ]:
num_classes = 5
import pandas as pd
truth_array = tf.argmax(pd.get_dummies(pd.Series(eval_generator.classes)),axis=1)
predictions_array = tf.argmax(predictions,axis=1)
print(truth_array)
print(predictions_array)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(truth_array, predictions_array)))
print(classification_report(truth_array,predictions_array))

file1 = open(r'/notebooks/Saved Plots/Accuracy_TestData/InceptionV3/InceptionV3_TSSL_V1_test.txt',"w")
file1.write(str((accuracy_score(truth_array, predictions_array))))
file1.write("\n")
file1.write(str(classification_report(truth_array,predictions_array)))
file1.close()

In [ ]:
print(max(history.history['val_cohen_kappa']))

print(max(history.history['val_specificity_at_sensitivity']))

In [ ]:
#compute conf mat
cf_matrix = tf.math.confusion_matrix(truth_array,predictions_array)
print(cf_matrix)

import seaborn as sns
fig, ax = plt.subplots()
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')
fig.savefig(r'/notebooks/Saved Plots/Confusion_Matrix/InceptionV3/InceptionV3_TSSL_V1_test')

In [ ]:
print(predictions)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
sick_vec = truth_array>0
sick_score = np.sum(predictions[:,1:],1)
fpr, tpr, _ = roc_curve(sick_vec, sick_score)
fig, ax1 = plt.subplots(1,1, figsize = (6, 6), dpi = 150)
ax1.plot(fpr, tpr, 'b.-', label = 'Model Prediction (AUC: %2.2f)' % roc_auc_score(sick_vec, sick_score))
ax1.plot(fpr, fpr, 'g-', label = 'Random Guessing')
ax1.legend()
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')


fig.savefig(r'/notebooks/Saved Plots/AUC_ROC/InceptionV3/InceptionV3_TSSL_V1_test')

In [ ]:
from sklearn.metrics import f1_score
f1_score(truth_array, predictions_array, average='weighted')